In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [39]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.storage.StorageLevel
import org.apache.spark.sql.functions._
import org.apache.spark.ml.evaluation.RegressionEvaluator 

val rootLogger = Logger.getRootLogger()
rootLogger.setLevel(Level.ERROR)

Logger.getLogger("org.apache.spark").setLevel(Level.WARN)
Logger.getLogger("org.spark-project").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}

import org.apache.spark.storage.StorageLevel

import org.apache.spark.sql.functions._

import org.apache.spark.ml.evaluation.RegressionEvaluator 


rootLogger: Logger = org.apache.log4j.spi.RootLogger@180481de

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.LinearRegression


val sparkSession = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/05/26 23:28:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/26 23:28:17 ERROR Shell: Failed to locate the winutils binary in the hadoop binary path
java.io.IOException: Could not locate executable null\bin\winutils.exe in the Hadoop binaries.
	at org.apache.hadoop.util.Shell.getQualifiedBinPath(Shell.java:378)
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:393)
	at org.apache.hadoop.util.Shell.<clinit>(Shell.java:386)
	at org.apache.hadoop.util.StringUtils.<clinit>(StringUtils.java:79)
	at org.apache.hadoop.security.Groups.parseStaticMapping(Groups.java:116)
	at org.apache.hadoop.security.Groups.<init>(Groups.java:93)
	at org.apache.hadoop.security.Groups.<init>(Groups.java:73)
	at org.apache.hadoop.security.Groups.getUserToGroupsMappingService(Groups.java:293)
	at org.apache.hadoop.security.UserGroupInform

Spark UI

import org.apache.spark.sql._

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.regression.LinearRegression



sparkSession: SparkSession = org.apache.spark.sql.SparkSession@46dd9775

In [4]:
val fireSchema = StructType(Array(StructField("Order ID", StringType),
                 StructField("Product", StringType, true),
                 StructField("Quantity Ordered", IntegerType, true),
                 StructField("Price Each", FloatType, true),
                 StructField("Order Date", StringType, true),
                 StructField("Purchase Address", StringType, true),
))

var annualDf:DataFrame=null

def loadCsvFile(fileName: String): DataFrame = {
    val pathName="csv/Sales_"+fileName+"_2019.csv"
  val df = sparkSession.read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .schema(fireSchema)
    .load(pathName)

  df
}
var dataframes: List[DataFrame]=List()
val months = List("January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December")
var i=0
months.foreach{(month)=>
           val df=loadCsvFile(month)
            if(annualDf==null) annualDf = df  else annualDf=annualDf.union(df)
}
val df=annualDf
df.cache()

fireSchema: StructType = StructType(
  StructField("Order ID", StringType, true, {}),
  StructField("Product", StringType, true, {}),
  StructField("Quantity Ordered", IntegerType, true, {}),
  StructField("Price Each", FloatType, true, {}),
  StructField("Order Date", StringType, true, {}),
  StructField("Purchase Address", StringType, true, {})
)
annualDf: DataFrame = [Order ID: string, Product: string ... 4 more fields]
defined function loadCsvFile
dataframes: List[DataFrame] = List()
months: List[String] = List(
  "January",
  "February",
  "March",
  "April",
  "May",
  "June",
  "July",
  "August",
  "September",
  "October",
  "November",
  "December"
)
i: Int = 0
df: DataFrame = [Order ID: string, Product: string ... 4 more fields]
res3_8: DataFrame = [Order ID: string, Product: string ... 4 more fields]

## 2.Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?
C'est le mois de Novembre avec un pic à 3715537.4414711 .

In [5]:
//2 Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?
val resultDf= df.groupBy(month(to_timestamp(col("Order Date"), "MM/dd/yyyy"))).agg(sum(col("Price Each") * col("Quantity Ordered")).as("TotalAmount")).orderBy(col("TotalAmount").desc)
                                   
 resultDf.show(1)

show at cmd4.sc:3

12 / 12

show at cmd4.sc:3

200 / 200

+-----------------------------------------------+---------------+
|month(to_timestamp(`Order Date`, 'MM/dd/yyyy'))|    TotalAmount|
+-----------------------------------------------+---------------+
|                                             11|3715537.4414711|
+-----------------------------------------------+---------------+
only showing top 1 row



resultDf: Dataset[Row] = [month(to_timestamp(`Order Date`, 'MM/dd/yyyy')): int, TotalAmount: double]

## 3.Quelle ville a enregistré le plus grand nombre de ventes ?
C'est la ville de San Francisco

In [13]:
// 3 Quelle ville a enregistré le plus grand nombre de ventes ?
val resultDf= df.groupBy(split(col("Purchase Address"),",")(1)).count().orderBy(col("count").desc)
                                   
 resultDf.show(1)

show at cmd12.sc:3

12 / 12

show at cmd12.sc:3

200 / 200

+-----------------------------+-----+
|split(Purchase Address, ,)[1]|count|
+-----------------------------+-----+
|                San Francisco|44732|
+-----------------------------+-----+
only showing top 1 row



resultDf: Dataset[Row] = [split(Purchase Address, ,)[1]: string, count: bigint]

## 4.À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients achètent le produit ?
Pour maximiser les chances que les clients achètent ils devraient diffuser des publicités entre 19h et 20h

In [7]:
//4 À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients
//achètent le produit ?
val resultDf= df.groupBy(hour(to_timestamp(col("Order Date"),"MM/dd/yyyy HH:mm"))).count().orderBy(col("count").desc)
                                   
 resultDf.show(1)

show at cmd6.sc:3

12 / 12

show at cmd6.sc:3 
 (kill)

199 + 1 / 200

+----------------------------------------------------+-----+
|hour(to_timestamp(`Order Date`, 'MM/dd/yyyy HH:mm'))|count|
+----------------------------------------------------+-----+
|                                                  19|12905|
+----------------------------------------------------+-----+
only showing top 1 row



resultDf: Dataset[Row] = [hour(to_timestamp(`Order Date`, 'MM/dd/yyyy HH:mm')): int, count: bigint]

## 5.Quels produits sont le plus souvent vendus ensemble ?
Voici les 10 produits les plus vendus ensemble

In [8]:
//5 Quels produits sont le plus souvent vendus ensemble ?
val transactionsDF = df.groupBy("Order ID").agg(collect_list("Product").as("produits"))
val explodedDF = transactionsDF.withColumn("product", explode(col("produits")))

val productPairsDF = explodedDF.alias("df1")
  .join(explodedDF.alias("df2"), col("df1.Order ID") === col("df2.Order ID") && col("df1.product") < col("df2.product"))
  .select(col("df1.product").as("product1"), col("df2.product").as("product2"))

val productPairCountsDF = productPairsDF.groupBy("product1", "product2")
  .count()
  .orderBy(desc("count"))

productPairCountsDF.show(10)

run at ThreadPoolExecutor.java:1149

12 / 12

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd7.sc:12

...

200 / 200

show at cmd7.sc:12

200 / 200

+--------------------+--------------------+-----+
|            product1|            product2|count|
+--------------------+--------------------+-----+
|Lightning Chargin...|              iPhone| 1015|
|        Google Phone|USB-C Charging Cable|  999|
|    Wired Headphones|              iPhone|  462|
|        Google Phone|    Wired Headphones|  423|
|Apple Airpods Hea...|              iPhone|  373|
|USB-C Charging Cable|     Vareebadd Phone|  368|
|Bose SoundSport H...|        Google Phone|  228|
|USB-C Charging Cable|    Wired Headphones|  205|
|     Vareebadd Phone|    Wired Headphones|  149|
|Lightning Chargin...|    Wired Headphones|  129|
+--------------------+--------------------+-----+
only showing top 10 rows



transactionsDF: DataFrame = [Order ID: string, produits: array<string>]
explodedDF: DataFrame = [Order ID: string, produits: array<string> ... 1 more field]
productPairsDF: DataFrame = [product1: string, product2: string]
productPairCountsDF: Dataset[Row] = [product1: string, product2: string ... 1 more field]

## 6.Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?
    Le produit le plus vendu est le cable de chargement usb-c. Ce résultat est compréhensible car c'est un cable adapté à plusieurs types d'appareils(smartphones,tablettes,ordinateurs) et à plusieurs systèmes d'exploitaion(android,apple): Ce qui fait sa popularité.


In [ ]:
// 6 Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?
val mostOrderedProduct= df.groupBy("Product").count().orderBy(col("count").desc)
mostOrderedProduct.show(5)

show at cmd61.sc:2 
 (kill)

0 + 1 / 12

## 7.Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?
Nous allons calculer la probabilité basée sur les tendances de ventes actuelle du dataset.

In [17]:
//7 Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?
val usbCount=df.filter(col("Product")==="USB-C Charging Cable").count().toDouble
val totalProducts=df.count().toDouble
val probabUsbC=(usbCount/totalProducts).toDouble
print(probabUsbC)

count at cmd16.sc:1

12 / 12

count at cmd16.sc:1

1 / 1

count at cmd16.sc:2

12 / 12

count at cmd16.sc:2

1 / 1

0.11722237088573723

usbCount: Double = 21903.0
totalProducts: Double = 186850.0
probabUsbC: Double = 0.11722237088573723

## 8 Quelle est la probabilité que les prochains clients commandent un iPhone ?

In [18]:
//8 Quelle est la probabilité que les prochains clients commandent un iPhone ?
val iphoneCount=df.filter(col("Product")==="iPhone").count().toDouble
val probabIphone=(iphoneCount/totalProducts).toDouble
print(probabIphone)

count at cmd17.sc:1

12 / 12

count at cmd17.sc:1

1 / 1

0.03661760770671662

iphoneCount: Double = 6842.0
probabIphone: Double = 0.03661760770671662

## 9.Quelle est la probabilité que les prochains clients commandent un téléphone Google ?

In [19]:
// 9 Quelle est la probabilité que les prochains clients commandent un téléphone Google ?
val googlePhoneCount=df.filter(col("Product")==="Google Phone").count().toDouble
val probabGooglePhone=(googlePhoneCount/totalProducts).toDouble
print(probabGooglePhone)

count at cmd18.sc:1

12 / 12

count at cmd18.sc:1

1 / 1

0.02956917313352957

googlePhoneCount: Double = 5525.0
probabGooglePhone: Double = 0.02956917313352957

## 10.Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?

In [20]:
// 10 Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?
val wiredHeadPhonesCount=df.filter(col("Product")==="Wired Headphones").count().toDouble
val probabWiredHeadPhones=(wiredHeadPhonesCount/totalProducts).toDouble
print(probabWiredHeadPhones)

count at cmd19.sc:1

12 / 12

count at cmd19.sc:1

1 / 1

0.10105432164838106

wiredHeadPhonesCount: Double = 18882.0
probabWiredHeadPhones: Double = 0.10105432164838106

# Proposition un cas d utilisation concret de Machine Learning
## Analyse et prévisions de la tendance des ventes en fonction de la période:
Cela va permettre à l'interprise d'avoir une connaissance sur la demande du marché et optimiser son stock en fonction. Ainsi, la satisfaction des clients sera améliorée et les ruptures et les pénuries de stock peuvent etre évitées. Ce qui va accroitre les gains de l'entreprise.

# Prepocessing des données

In [21]:
df.show()

show at cmd20.sc:1

1 / 1

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  141234|              iPhone|               1|     700.0|01/22/19 21:25|944 Walnut St, Bo...|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|
|  141236|    Wired Headphones|               2|     11.99|01/17/19 13:33|538 Adams St, San...|
|  141237|    27in FHD Monitor|               1|    149.99|01/05/19 20:33|738 10th St, Los ...|
|  141238|    Wired Headphones|               1|     11.99|01/25/19 11:59|387 10th St, Aust...|
|  141239|AAA Batteries (4-...|               1|      2.99|01/29/19 20:22|775 Willow St, Sa...|
|  141240|27in 4K Gaming Mo...|               1|    389.99|01/26/19 12:16|979 Park St, Los ...|
|  141241|USB-C Charging Cable|         

## Calcul du nombre de données manquantes dans chaque colonne

In [22]:
val missingCounts = annualDf.agg(
  sum(when(col("Order ID").isNull || col("Order ID").isNaN, 1).otherwise(0)).alias("missing_order"),
  sum(when(col("Product").isNull || col("Product").isNaN, 1).otherwise(0)).alias("missing_product"),
  sum(when(col("Quantity Ordered").isNull || col("Quantity Ordered").isNaN, 1).otherwise(0)).alias("missing_qte"),
  sum(when(col("Price Each").isNull || col("Price Each").isNaN, 1).otherwise(0)).alias("missing_price"),
  sum(when(col("Order Date").isNull || col("Order Date").isNaN, 1).otherwise(0)).alias("missing_date"),
  sum(when(col("Purchase Address").isNull || col("Purchase Address").isNaN, 1).otherwise(0)).alias("missing_adress")
)

// Affichage des résultats
missingCounts.show()

show at cmd21.sc:11

12 / 12

show at cmd21.sc:11

1 / 1

+-------------+---------------+-----------+-------------+------------+--------------+
|missing_order|missing_product|missing_qte|missing_price|missing_date|missing_adress|
+-------------+---------------+-----------+-------------+------------+--------------+
|          900|            900|        900|          900|         900|           900|
+-------------+---------------+-----------+-------------+------------+--------------+



missingCounts: DataFrame = [missing_order: bigint, missing_product: bigint ... 4 more fields]

## suppression des donnees manquantes
Nous avons 900 données manquantes. Ce qui est négligeable comparé au volume de notre dataset(plus de 186000)

In [23]:
val cleanedData = annualDf.na.drop()
 cleanedData.show()

show at cmd22.sc:2

1 / 1

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  141234|              iPhone|               1|     700.0|01/22/19 21:25|944 Walnut St, Bo...|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|
|  141236|    Wired Headphones|               2|     11.99|01/17/19 13:33|538 Adams St, San...|
|  141237|    27in FHD Monitor|               1|    149.99|01/05/19 20:33|738 10th St, Los ...|
|  141238|    Wired Headphones|               1|     11.99|01/25/19 11:59|387 10th St, Aust...|
|  141239|AAA Batteries (4-...|               1|      2.99|01/29/19 20:22|775 Willow St, Sa...|
|  141240|27in 4K Gaming Mo...|               1|    389.99|01/26/19 12:16|979 Park St, Los ...|
|  141241|USB-C Charging Cable|         

cleanedData: DataFrame = [Order ID: string, Product: string ... 4 more fields]

## Vérification après suppression

In [24]:
val missingCountsAfterClean = cleanedData.agg(
  sum(when(col("Order ID").isNull || col("Order ID").isNaN, 1).otherwise(0)).alias("missing_order"),
  sum(when(col("Product").isNull || col("Product").isNaN, 1).otherwise(0)).alias("missing_product"),
  sum(when(col("Quantity Ordered").isNull || col("Quantity Ordered").isNaN, 1).otherwise(0)).alias("missing_qte"),
  sum(when(col("Price Each").isNull || col("Price Each").isNaN, 1).otherwise(0)).alias("missing_price"),
  sum(when(col("Order Date").isNull || col("Order Date").isNaN, 1).otherwise(0)).alias("missing_date"),
  sum(when(col("Purchase Address").isNull || col("Purchase Address").isNaN, 1).otherwise(0)).alias("missing_adress")
)
 missingCountsAfterClean.show()

show at cmd23.sc:9

12 / 12

show at cmd23.sc:9

1 / 1

+-------------+---------------+-----------+-------------+------------+--------------+
|missing_order|missing_product|missing_qte|missing_price|missing_date|missing_adress|
+-------------+---------------+-----------+-------------+------------+--------------+
|            0|              0|          0|            0|           0|             0|
+-------------+---------------+-----------+-------------+------------+--------------+



missingCountsAfterClean: DataFrame = [missing_order: bigint, missing_product: bigint ... 4 more fields]

## Agrégation des données de ventes en fonction du mois

In [34]:
val MonthLyData=cleanedData.groupBy(month(to_timestamp(col("Order Date"),"MM/dd/yyyy")).alias("month")).agg(sum(col("Price Each") * col("Quantity Ordered")).alias("salesAmount")).orderBy(col("month"))
MonthLyData.show()

show at cmd33.sc:2

12 / 12

show at cmd33.sc:2

200 / 200

+-----+------------------+
|month|       salesAmount|
+-----+------------------+
|    1|2065599.3996958733|
|    2|2345457.7183713913|
|    3|3043415.1442615986|
|    4| 3320523.313942671|
|    5|2943146.8557178974|
|    6|2553346.2173449993|
|    7| 2532465.747330904|
|    8| 2192137.809127569|
|    9| 2674799.235773325|
|   10| 3587664.421559572|
|   11|   3715537.4414711|
|   12| 3517942.491746187|
+-----+------------------+



MonthLyData: Dataset[Row] = [month: int, salesAmount: double]

# Modélisation

## Séparation entre données de test et données d entrainement

In [35]:
val Array(trainData, testData) = MonthLyData.randomSplit(Array(0.8, 0.2), seed = 42)

trainData: Dataset[Row] = [month: int, salesAmount: double]
testData: Dataset[Row] = [month: int, salesAmount: double]

## Préparation des données pour l entraînement du modèle de régression linéaire

In [36]:
val assembler = new VectorAssembler()
  .setInputCols(Array("month"))
  .setOutputCol("features")
val assembledData = assembler.transform(trainData)


assembler: VectorAssembler = vecAssembler_58fbdfbf255f
assembledData: DataFrame = [month: int, salesAmount: double ... 1 more field]

## Entraînement du modèle

In [61]:
val lr = new LinearRegression()
  .setLabelCol("salesAmount")
.setFeaturesCol("features")

val lrModel = lr.fit(assembledData)

first at LinearRegression.scala:322

9 + 3 / 12

Cancelling stage 116


: 

## coefficients et intercept du modèle

In [38]:
println(s"Coefficients: ${lrModel.coefficients}")
println(s"Intercept: ${lrModel.intercept}")


Coefficients: [107885.0780122118]
Intercept: 2122755.223881863


## Prédictions

In [44]:
// Faire des prédictions sur les données de test
val testDataFeatures = assembler.transform(testData)
val testPredictions = lrModel.transform(testDataFeatures)

// Afficher les prédictions
// predictions.select("month", "prediction").show()
// Faire des prévisions pour les prochains mois
val futureMonths = sparkSession.range(13, 24).toDF("month")
val futureData = assembler.transform(futureMonths)
val predictions = lrModel.transform(futureData)

rdd at RegressionEvaluator.scala:81

12 / 12

treeAggregate at RegressionMetrics.scala:57

200 / 200

treeAggregate at RegressionMetrics.scala:57

13 / 13

Root Mean Squared Error (RMSE): 422174.5212777637


testDataFeatures: DataFrame = [month: int, salesAmount: double ... 1 more field]
testPredictions: DataFrame = [month: int, salesAmount: double ... 2 more fields]
evaluator: RegressionEvaluator = regEval_06471b361c7c
rmse: Double = 422174.5212777637
futureMonths: DataFrame = [month: bigint]
futureData: DataFrame = [month: bigint, features: vector]
predictions: DataFrame = [month: bigint, features: vector ... 1 more field]

## Evaluation

In [47]:
// Évaluation des performances du modèle
val evaluator = new RegressionEvaluator()
  .setLabelCol("salesAmount")
  .setPredictionCol("prediction")
  .setMetricName("rmse")

val rmse = evaluator.evaluate(testPredictions)
println(s"Root Mean Squared Error (RMSE): $rmse")


rdd at RegressionEvaluator.scala:81

12 / 12

treeAggregate at RegressionMetrics.scala:57

200 / 200

treeAggregate at RegressionMetrics.scala:57

13 / 13

Root Mean Squared Error (RMSE): 422174.5212777637


evaluator: RegressionEvaluator = regEval_a05845647aca
rmse: Double = 422174.5212777637

In [45]:
// Afficher les prévisions pour les prochains mois
predictions.show()

show at cmd44.sc:1

1 / 1

show at cmd44.sc:1

3 / 3

+-----+--------+------------------+
|month|features|        prediction|
+-----+--------+------------------+
|   13|  [13.0]|3525261.2380406167|
|   14|  [14.0]| 3633146.316052828|
|   15|  [15.0]|  3741031.39406504|
|   16|  [16.0]| 3848916.472077252|
|   17|  [17.0]|3956801.5500894636|
|   18|  [18.0]| 4064686.628101676|
|   19|  [19.0]| 4172571.706113887|
|   20|  [20.0]| 4280456.784126099|
|   21|  [21.0]|  4388341.86213831|
|   22|  [22.0]| 4496226.940150523|
|   23|  [23.0]| 4604112.018162735|
+-----+--------+------------------+



In [57]:
// Extraire les mois et les ventes en tant que listes
val monthsSeq = Seq("Jan", "Fév", "Mar", "Avr", "Mai", "Juin", "Juil", "Aout", "Sep", "Oct", "Nov", "Déc")
val sales = MonthLyData.collect().map(_.getDouble(1))
val predSales = predictions.collect().map(_.getDouble(2))


collect at cmd56.sc:2

12 / 12

collect at cmd56.sc:3

4 / 4

monthsSeq: Seq[String] = List(
  "Jan",
  "F\u00e9v",
  "Mar",
  "Avr",
  "Mai",
  "Juin",
  "Juil",
  "Aout",
  "Sep",
  "Oct",
  "Nov",
  "D\u00e9c"
)
sales: Array[Double] = Array(
  2065599.3996958733,
  2345457.7183713913,
  3043415.1442615986,
  3320523.313942671,
  2943146.8557178974,
  2553346.2173449993,
  2532465.747330904,
  2192137.809127569,
  2674799.235773325,
  3587664.421559572,
  3715537.4414711,
  3517942.491746187
)
predSales: Array[Double] = Array(
  3525261.2380406167,
  3633146.316052828,
  3741031.39406504,
  3848916.472077252,
  3956801.5500894636,
  4064686.628101676,
  4172571.706113887,
  4280456.784126099,
  4388341.86213831,
  4496226.940150523,
  4604112.018162735
)

In [50]:

//Diagramme en barres
val barChart = Bar(
  x = monthsSeq,
  y = sales.toSeq
)

barChart: Bar = Bar(
  Strings(
    List(
      "Jan",
      "F\u00e9v",
      "Mar",
      "Avr",
      "Mai",
      "Juin",
      "Juil",
      "Aout",
      "Sep",
      "Oct",
      "Nov",
      "D\u00e9c"
    )
  ),
  Doubles(
    WrappedArray(
      2065599.3996958733,
      2345457.7183713913,
      3043415.1442615986,
      3320523.313942671,
      2943146.8557178974,
      2553346.2173449993,
      2532465.747330904,
      2192137.809127569,
      2674799.235773325,
      3587664.421559572,
      3715537.4414711,
      3517942.491746187
    )
  ),
  None,
  None,
  None,
  None,
  None,
  None,
...

In [51]:
// Créer la mise en page du graphique
val layout = Layout(
  title = "Tendances de vente mensuelles",
  xaxis = Axis(title = "Mois"),
  yaxis = Axis(title = "Ventes")
)

layout: Layout = Layout(
  Some("Tendances de vente mensuelles"),
  None,
  None,
  None,
  None,
  Some(
    Axis(
      Some("Mois"),
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
...

In [52]:
// Afficher le graphique
plot(Seq(barChart), layout)

res51: String = "plot-cea49c80-eedc-45b8-b830-3ac7d7cb5dbc"

In [58]:
// Créer un graphique linéaire avec les mois en tant que x et les ventes en tant que y
val lineChart1 = Scatter(
  x = monthsSeq,
  y = sales.toSeq,
)
val lineChart2 = Scatter(
  x = monthsSeq,
  y = predSales.toSeq,

)

lineChart1: Scatter = Scatter(
  Some(
    Strings(
      List(
        "Jan",
        "F\u00e9v",
        "Mar",
        "Avr",
        "Mai",
        "Juin",
        "Juil",
        "Aout",
        "Sep",
        "Oct",
        "Nov",
        "D\u00e9c"
      )
    )
  ),
  Some(
    Doubles(
      WrappedArray(
        2065599.3996958733,
        2345457.7183713913,
        3043415.1442615986,
        3320523.313942671,
        2943146.8557178974,
        2553346.2173449993,
        2532465.747330904,
        2192137.809127569,
        2674799.235773325,
        3587664.421559572,
        3715537.4414711,
        3517942.491746187
      )
    )
  ),
  None,
  None,
...
lineChart2: Scatter = Scatter(
  Some(
    Strings(
      List(
        "Jan",
        "F\u00e9v",
        "Mar",
        "Avr",
        "Mai",
        "Juin",
        "Juil",
        "Aout",
        "Sep",
        "Oct",
        "Nov",
        "D\u00e9c"
      )
    )
  ),
  Some(
    Doubles(
      WrappedArray(
   

In [54]:
// Créer la mise en page du graphique
val layout = Layout(
  title = "Tendances de vente mensuelles",
  xaxis = Axis(title = "Mois"),
  yaxis = Axis(title = "Ventes"),
)

layout: Layout = Layout(
  Some("Tendances de vente mensuelles"),
  None,
  None,
  None,
  None,
  Some(
    Axis(
      Some("Mois"),
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
      None,
...

In [60]:
// Afficher le graphique
plot(Seq(lineChart1), layout)
plot(Seq(lineChart2), layout)

res59_0: String = "plot-cd95ff7a-582e-4b49-a998-f2cee825277a"
res59_1: String = "plot-28d94c03-4c5f-4bec-8e9c-42a03eb765c2"

In [ ]:
val model=ARIMA.fitModel(sales)